# Handwrite Digits Recognition (MNIST)
This notebook tends to build up a neutral network which can receive a 28*28 gray handwritten digits pic and output the digit on the pic.
Provided a simple classification solution and CNN solution.



## Environment & Datasets
Set up necessary env.

In [11]:
import torch
from ipywidgets import interact
import ipywidgets as widgets
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
batch_size = 64

In [ ]:
# 创建一个滑块
slider = widgets.FloatSlider(
    value=0.0,   # 初始值
    min=-1.0,    # 最小值
    max=1.0,     # 最大值
    step=0.1,    # 步长
    description='Slider:',  # 描述文本
    orientation='horizontal'  # 方向
)

# 显示滑块
display(slider)

Set up training and test dataset. Then define dataloader

In [2]:
trainingData = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

testData = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(trainingData, batch_size=batch_size)
test_dataloader = DataLoader(testData, batch_size=batch_size)

100.0%


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100.0%


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100.0%


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100.0%

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



Load computing device.


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## Neutral Network
Provided 2 solution.
1. Only classification layer.
2. CNN

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
         # 特征提取网络
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        self.pool1 = nn.MaxPool2d(2) 
        self.drop1 = nn.Dropout(p=0.15)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=2)
        self.pool2 = nn.MaxPool2d(2)
        self.drop2 = nn.Dropout(p=0.15)
        
        # 分类网络
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(576, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        self.fc1 = nn.Linear(576, 64)
        self.fc2 = nn.Linear(64, 10)
    # 前向传播
    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.drop1(self.pool1(x))
        x = nn.functional.relu(self.conv2(x))
        x = self.drop2(self.pool2(x))
 
        x = torch.flatten(x, start_dim=1)
        
        logits = self.linear_relu_stack(x)
        
        # softmax = nn.Softmax(dim=1)(logits)
        # return softmax
        return logits
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
       
        return x
    
model = CNN().to(device)
print(model)

## Train

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            writer.add_scalar("Loss/train", loss, current)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

In [9]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.001432  [   64/60000]
loss: 0.000393  [ 6464/60000]
loss: 0.077949  [12864/60000]
loss: 0.049390  [19264/60000]
loss: 0.005559  [25664/60000]
loss: 0.003176  [32064/60000]
loss: 0.012386  [38464/60000]
loss: 0.022230  [44864/60000]
loss: 0.077024  [51264/60000]
loss: 0.002221  [57664/60000]
Test Error: 
 Accuracy: 97.5%, Avg loss: 0.124765 

Epoch 2
-------------------------------
loss: 0.021014  [   64/60000]
loss: 0.000202  [ 6464/60000]
loss: 0.000580  [12864/60000]
loss: 0.001408  [19264/60000]
loss: 0.028782  [25664/60000]
loss: 0.012704  [32064/60000]
loss: 0.000856  [38464/60000]
loss: 0.002073  [44864/60000]
loss: 0.120290  [51264/60000]
loss: 0.000713  [57664/60000]
Test Error: 
 Accuracy: 97.6%, Avg loss: 0.121407 

Epoch 3
-------------------------------
loss: 0.001369  [   64/60000]
loss: 0.004459  [ 6464/60000]
loss: 0.007184  [12864/60000]
loss: 0.001189  [19264/60000]
loss: 0.000918  [25664/60000]
loss: 0.002873  [32064/600